<a href="https://colab.research.google.com/github/ejngotho1/Amazon_Vine_Analysis/blob/main/tokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
# creating sample df
dataframe = spark.createDataFrame([
                                   (0,"Spark rocks"),
                                   (1,"we are learning now"),
                                   (2,"Spark is much better")
],["id","sentence"])
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|         Spark rocks|
|  1| we are learning now|
|  2|Spark is much better|
+---+--------------------+



In [ ]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_940f5270ed68

In [ ]:
# transform and show
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate = False)

+---+--------------------+-------------------------+
|id |sentence            |words                    |
+---+--------------------+-------------------------+
|0  |Spark rocks         |[spark, rocks]           |
|1  |we are learning now |[we, are, learning, now] |
|2  |Spark is much better|[spark, is, much, better]|
+---+--------------------+-------------------------+



In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [ ]:
# create our tokenizer
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# Transform dataframe
tokenized_df = tokenizer.transform(dataframe)

# select needed cols and don't truncate
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+--------------------+-------------------------+------+
|id |sentence            |words                    |tokens|
+---+--------------------+-------------------------+------+
|0  |Spark rocks         |[spark, rocks]           |2     |
|1  |we are learning now |[we, are, learning, now] |4     |
|2  |Spark is much better|[spark, is, much, better]|4     |
+---+--------------------+-------------------------+------+

